<a href="https://colab.research.google.com/github/shakirabanu20/Movie-recommendation-/blob/main/Movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Install required packages
!pip install gradio pandas scikit-learn --quiet

# STEP 2: Upload your CSV file
from google.colab import files
uploaded = files.upload()  # Upload your properly formatted movie_dataset.csv here

# STEP 3: Import required libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr

# STEP 4: Load dataset with error handling for malformed rows
df = pd.read_csv("movies_data.csv", encoding='utf-8', on_bad_lines='skip')

# Normalize movie titles for case-insensitive matching
df['title_lower'] = df['title'].str.lower()

# STEP 5: Create TF-IDF matrix based on genres column
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['genres'])

# Calculate cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Create a reverse mapping from title_lower to dataframe index
indices = pd.Series(df.index, index=df['title_lower']).drop_duplicates()

# STEP 6: Define recommendation function
def recommend_movie(title):
    title_lower = title.lower().strip()
    if title_lower not in indices:
        return "❌ Movie not found. Please try another title."

    idx = indices[title_lower]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    movie_indices = [i[0] for i in sim_scores]

    result = ""
    for i in movie_indices:
        result += f"🎬 **{df['title'].iloc[i]}**\n"
        result += f"⭐ Rating: {df['rating'].iloc[i]}/10\n"
        result += f"🎭 Genre: {df['genres'].iloc[i]}\n"
        result += f"📝 Tagline: _{df['tagline'].iloc[i]}_\n\n"
    return result.strip()

# STEP 7: Launch Gradio app
gr.Interface(
    fn=recommend_movie,
    inputs=gr.Textbox(lines=1, label="🎥 Enter a Movie Title"),
    outputs=gr.Markdown(label="🎯 Recommended Movies with Details"),
    title="🎬 AI Movie Recommender",
    description="Enter a movie title to get personalized recommendations including ratings and taglines."
).launch(share=True)